In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
from darts.timeseries import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler

In [3]:
df = pd.read_csv("nyc_data.csv", index_col=0, parse_dates=True)
df = df.rename(columns={"Demand":"y"})
df.head(1)

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305


In [4]:
series = TimeSeries.from_series(df.y)
covariates = TimeSeries.from_dataframe(df.iloc[:, 1:])

In [8]:
year_series = datetime_attribute_timeseries(pd.date_range(start=series.start_time(),
                                            freq=series.freq_str,
                                            periods=df.shape[0]),
                                            attribute='year',
                                            one_hot=False)
month_series = datetime_attribute_timeseries(year_series,
                                             attribute='month',
                                             one_hot=True)
weekday_series = datetime_attribute_timeseries(year_series,
                                               attribute='weekday',
                                               one_hot=True)

In [33]:
# Scaling
transformer1 = Scaler()
transformer2 = Scaler()

In [34]:
y_transformed = transformer1.fit_transform(series)
covariates = covariates.stack(year_series)
covariates_transformed = transformer2.fit_transform(covariates)

In [35]:
covariates_transformed = covariates_transformed.stack(month_series)
covariates_transformed = covariates_transformed.stack(weekday_series)

### LSTM Model

In [16]:
from darts.models import RNNModel

In [48]:
model = RNNModel(model='LSTM',
                 hidden_dim=20,
                 n_rnn_layers=2,
                 dropout=0.2,
                 n_epochs=10,
                 optimizer_kwargs={'lr':0.003},
                 random_state=42,
                 training_length=20,
                 input_chunk_length=15,
                 pl_trainer_kwargs = {"accelerator": "auto"})

In [49]:
model.fit(y_transformed,
          future_covariates=covariates_transformed)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 7.3 K 
4 | V             | Linear           | 21    
---------------------------------------------------
7.3 K     Trainable params
0         Non-trainable params
7.3 K     Total params
0.029     Total estimated model params size (MB)


Epoch 9:  35%|███▌      | 24/68 [00:00<00:00, 56.40it/s, train_loss=0.00317]

c:\Users\Yixiong Lin\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


RNNModel(model=LSTM, hidden_dim=20, n_rnn_layers=2, dropout=0.2, training_length=20, n_epochs=10, optimizer_kwargs={'lr': 0.003}, random_state=42, input_chunk_length=15, pl_trainer_kwargs={'accelerator': 'auto'})

#### Cross Validation

In [30]:
cv = model.historical_forecasts(series=y_transformed,
                                future_covariates=covariates_transformed,
                                start=df.shape[0]-180,
                                forecast_horizon=31,
                                stride=16,
                                retrain=True,
                                last_points_only=False) 

`enable_optimization=True` is ignored because `retrain` is not `False` or `0`.To hide this warning, set `show_warnings=False` or `enable_optimization=False`.
`enable_optimization=True` is ignored because `forecast_horizon > model.output_chunk_length`.To hide this warning, set `show_warnings=False` or `enable_optimization=False`.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 7.2 K 
4 | V             | Linear           | 21    
---------------------------------------------------
7.2 K     Trainable params
0         Non-trainable params
7.2 K     Total params
0.029     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 63/63 [00:01<00:00, 57.64it/s, train_loss=0.00277]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 63/63 [00:01<00:00, 57.53it/s, train_loss=0.00277]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 7.2 K 
4 | V             | Linear           | 21    
---------------------------------------------------
7.2 K     Trainable params
0         Non-trainable params
7.2 K     Total params
0.029     Total estimated model params size (MB)


...

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 7.2 K 
4 | V             | Linear           | 21    
---------------------------------------------------
7.2 K     Trainable params
0         Non-trainable params
7.2 K     Total params
0.029     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 67/67 [00:01<00:00, 51.09it/s, train_loss=0.00319]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 67/67 [00:01<00:00, 51.05it/s, train_loss=0.00319]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [37]:
from sklearn.metrics import mean_squared_error
# Store the results
rmse_cv = []

for i in range(len(cv)):

  # Compute the RMSE for the CV
  predictions = TimeSeries.pd_series(transformer1.inverse_transform(cv[i]))

  # Actual values
  start = predictions.index.min()
  end = predictions.index.max()
  actuals = df.y[start:end]

  #compute the error
  error_cv = mean_squared_error(actuals, predictions, squared = False)

  # save the error
  rmse_cv.append(error_cv)

print(f"The RMSE is {np.mean(rmse_cv)}")

The RMSE is 86.65470872218272


#### Parameter Tuning

In [39]:
# Grid
param_grid = {'n_rnn_layers': [1,2],
              'hidden_dim': [10,20],
              'dropout': [0.1,0.2],
              'n_epochs': [10,20],
              'lr': [0.003],
              'training_length': [20],
              'input_chunk_length': [15]}
grid = ParameterGrid(param_grid)
len(list(grid))

16

In [50]:
import torch

In [51]:
torch.cuda.is_available()

False